<a href="https://colab.research.google.com/github/nvanandsah/OpenCL_VGG16/blob/master/Python/OpenCL_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# New Section

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import keras 
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.layers import Dense, Embedding, SimpleRNN, Input, Conv2D, Flatten, MaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

from __future__ import print_function

from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.models import Sequential
from keras import optimizers

from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

from keras.applications.vgg19 import VGG19

Using TensorFlow backend.


In [ ]:
# load the model
model = VGG16()
# load an image from file
image = load_img('/content/mug.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

40960/35363 [==================================] - 0s 0us/step
espresso (57.41%)


In [ ]:
print(model.summary())
network_layers = [layer.name for layer in model.layers]
print(network_layers)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
feature_model = Model(
    model.inputs,
    [layer.output for layer in model.layers]
)
out = feature_model.predict(image)

decode_predictions(out[-1])

[[('n07920052', 'espresso', 0.57408124),
  ('n03063599', 'coffee_mug', 0.24631496),
  ('n07930864', 'cup', 0.11509778),
  ('n03297495', 'espresso_maker', 0.010166128),
  ('n04398044', 'teapot', 0.0067943865)]]

In [ ]:
out_dict = {}
for i in range(len(out)):
  out_dict[network_layers[i]] = out[i]


for key in out_dict.keys():
  print(key,out_dict[key].shape)

for index, layer in enumerate(model.layers):
  #if(isinstance(layer,Conv2D) or isinstance(layer,Dense) or isinstance(layer,MaxPooling2D) or isinstance(layer,BatchNormalization)):
  print(index, (layer.name))

input_2 (1, 224, 224, 3)
block1_conv1 (1, 224, 224, 64)
block1_conv2 (1, 224, 224, 64)
block1_pool (1, 112, 112, 64)
block2_conv1 (1, 112, 112, 128)
block2_conv2 (1, 112, 112, 128)
block2_pool (1, 56, 56, 128)
block3_conv1 (1, 56, 56, 256)
block3_conv2 (1, 56, 56, 256)
block3_conv3 (1, 56, 56, 256)
block3_pool (1, 28, 28, 256)
block4_conv1 (1, 28, 28, 512)
block4_conv2 (1, 28, 28, 512)
block4_conv3 (1, 28, 28, 512)
block4_pool (1, 14, 14, 512)
block5_conv1 (1, 14, 14, 512)
block5_conv2 (1, 14, 14, 512)
block5_conv3 (1, 14, 14, 512)
block5_pool (1, 7, 7, 512)
flatten (1, 25088)
fc1 (1, 4096)
fc2 (1, 4096)
predictions (1, 1000)
0 input_2
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten
20 fc1
21 fc2
22 predictions


In [ ]:
file_location = "drive/My Drive/OpenCL/VGGweights/"

for index, layer in enumerate(model.layers):
  if(isinstance(layer,Conv2D)):
    print(index, layer.name)
    #fileName = file_location + layer.name + ".txt"
    fileName = file_location + str(index) + "Conv2d" + ".txt"
    fileHandler = open(fileName, "w")
    fileHandler.write(model.layers[index].name)
    fileHandler.write("\n")
    kernelWidth = model.layers[index].get_weights()[0].shape[0]
    kernelHeight = model.layers[index].get_weights()[0].shape[1]
    kernelDepth = model.layers[index].get_weights()[0].shape[2]
    numChannels = model.layers[index].get_weights()[0].shape[3]
    # Write kernel details
    fileHandler.write(str(kernelWidth) + " " + str(kernelHeight) + " " + str(kernelDepth) + " " + str(numChannels) + "\n")
    # Store weights
    for channels in range(0, numChannels):
      for width in range(kernelWidth):
        for height in range(kernelHeight):
          for depth in range(kernelDepth):
            fileHandler.write(str(model.layers[index].get_weights()[0][width, height, depth, channels]) + " ")
    fileHandler.write("\n")
    # Store biases
    for channels in range(0, numChannels):
      fileHandler.write(str(model.layers[index].get_weights()[1][channels]) + " ");
    fileHandler.close()

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten
20 fc1
21 fc2
22 predictions
1 block1_conv1
2 block1_conv2
4 block2_conv1
5 block2_conv2
7 block3_conv1
8 block3_conv2
9 block3_conv3
11 block4_conv1
